<a href="https://colab.research.google.com/github/gocgodman/M2M/blob/main/M2M(mp3_to_midi).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install note-seq librosa soundfile pretty_midi
!pip install magenta==2.1.3 --no-deps

  Using cached note_seq-0.0.5-py3-none-any.whl.metadata (1.1 kB)
  Using cached pretty_midi-0.2.11.tar.gz (5.6 MB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.11-py3-none-any.whl size=5595886 sha256=fe94e92f896a444ff0d2c0db5420577c450f7178f998b34e2d5b5167759470ff
  Stored in directory: /root/.cache/pip/wheels/f4/ad/93/a7042fe12668827574927ade9deec7f29aad2a1001b1501882
Successfully built pretty_midi
  Using cached magenta-2.1.3-py3-none-any.whl.metadata (2.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.5 MB/s eta 0:00:00


In [ ]:

from google.colab import files

uploaded = files.upload()

INPUT = list(uploaded.keys())[0]  # ← 핵심
print("입력 파일:", INPUT)

Saving 01_Flower_Dance_-_DJ_Okawari_ver.mp3 to 01_Flower_Dance_-_DJ_Okawari_ver (2).mp3
입력 파일: 01_Flower_Dance_-_DJ_Okawari_ver (2).mp3


In [ ]:
import librosa
import numpy as np
import soundfile as sf

CLEAN_WAV = "piano_clean.wav"

y, sr = librosa.load(INPUT, sr=16000, mono=True)

# STFT
n_fft = 2048
hop = 512
S = librosa.stft(y, n_fft=n_fft, hop_length=hop)
mag, phase = np.abs(S), np.angle(S)

# dB 변환
mag_db = librosa.amplitude_to_db(mag, ref=np.max)

# ===== 파라미터 =====
REL_DB = -40          # 상대 dB 컷 (기본값)
TOP_K = 10            # 프레임당 상위 주파수 개수
LOW_FREQ_CUT = 40     # 저주파 노이즈 컷
HIGH_FREQ_PROTECT = 3000  # 고음 보호
# ====================

freqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)[:, None]

# 프레임별 최대 dB
frame_max = mag_db.max(axis=0, keepdims=True)

# 1️⃣ 상대 dB 마스크
mask_db = mag_db >= (frame_max + REL_DB)

# 2️⃣ 프레임당 상위 K 주파수
topk_mask = np.zeros_like(mag, dtype=bool)
for t in range(mag.shape[1]):
    idx = np.argsort(mag[:, t])[-TOP_K:]
    topk_mask[idx, t] = True

# 3️⃣ 저주파 제거
mask_low = freqs > LOW_FREQ_CUT

# 4️⃣ 고음 보호
mask_high = freqs > HIGH_FREQ_PROTECT

# 최종 마스크
final_mask = (mask_db & topk_mask & mask_low) | mask_high

# 적용
mag_clean = mag * final_mask
S_clean = mag_clean * np.exp(1j * phase)

# 복원
y_clean = librosa.istft(S_clean, hop_length=hop)

# 저장
sf.write(CLEAN_WAV, y_clean, sr)
print("전처리 완료:", CLEAN_WAV)

전처리 완료: piano_clean.wav


In [ ]:

!mkdir -p onsets_frames_checkpoint
!wget -O onsets_frames_checkpoint/model.ckpt.data-00000-of-00001 \
https://storage.googleapis.com/magentadata/models/onsets_frames_transcription/maestro_checkpoint/model.ckpt.data-00000-of-00001

!wget -O onsets_frames_checkpoint/model.ckpt.index \
https://storage.googleapis.com/magentadata/models/onsets_frames_transcription/maestro_checkpoint/model.ckpt.index

!wget -O onsets_frames_checkpoint/model.ckpt.meta \
https://storage.googleapis.com/magentadata/models/onsets_frames_transcription/maestro_checkpoint/model.ckpt.meta

--2026-01-15 08:47:11--  https://storage.googleapis.com/magentadata/models/onsets_frames_transcription/maestro_checkpoint/model.ckpt.data-00000-of-00001
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 172.253.118.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2026-01-15 08:47:11 ERROR 404: Not Found.

--2026-01-15 08:47:11--  https://storage.googleapis.com/magentadata/models/onsets_frames_transcription/maestro_checkpoint/model.ckpt.index
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 172.253.118.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2026-01-15 08:47:12 ERROR 404: Not Found.

--2026-01-15 08:47:12--  https://storage.googleapis.com/magentadata/models/onsets_frames_transcr

In [ ]:
from magenta.models.onsets_frames_transcription import infer_util
from magenta.models.onsets_frames_transcription import configs

OUTPUT_MIDI = "piano.mid"

audio, sr = librosa.load(CLEAN_WAV, sr=16000, mono=True)

config = configs.CONFIG_MAP['onsets_frames']
infer_util.transcribe(
    audio=audio,
    sample_rate=sr,
    checkpoint_path="onsets_frames_checkpoint",
    output_midi_path=OUTPUT_MIDI
)

print("MIDI 생성 완료:", OUTPUT_MIDI)

In [ ]:
files.download("piano.mid")

In [ ]:
# ================================
# 🔥 ALL-IN-ONE mp3 → MIDI + 평가 + Gradio
# ================================

# ---------- 설치 ----------
!apt-get update -qq
!apt-get install -qq libfluidsynth1 fluid-soundfont-gm
!pip install -q librosa soundfile pretty_midi gradio
!pip install -qU magenta

# ---------- 체크포인트 ----------
!rm -rf /content/onsets-frames
!mkdir /content/onsets-frames
!gsutil -q -m cp -R gs://magentadata/models/onsets_frames_transcription/*checkpoint.zip /content/onsets-frames/
!unzip -o /content/onsets-frames/maestro_checkpoint.zip -d /content/onsets-frames/
CHECKPOINT_DIR = "/content/onsets-frames/maestro/train"

# ---------- imports ----------
import librosa, numpy as np, soundfile as sf, pretty_midi, gradio as gr
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from magenta.models.onsets_frames_transcription import configs, data, train_util, infer_util
from note_seq import midi_io

# ---------- 모델 초기화 ----------
config = configs.CONFIG_MAP['onsets_frames']
hparams = config.hparams
hparams.batch_size = 1
hparams.use_cudnn = False

examples = tf.placeholder(tf.string, [None])
dataset = data.provide_batch(
    examples=examples,
    preprocess_examples=True,
    params=hparams,
    is_training=False,
    shuffle_examples=False
)

estimator = train_util.create_estimator(
    config.model_fn, CHECKPOINT_DIR, hparams)

# ---------- 자동 평가 함수 ----------
def score_midi(midi_path):
    pm = pretty_midi.PrettyMIDI(midi_path)
    notes = [n for inst in pm.instruments for n in inst.notes]
    if not notes:
        return 0.0

    durations = np.array([n.end - n.start for n in notes])
    pitches = np.array([n.pitch for n in notes])

    short_penalty = np.mean(durations < 0.05)
    pitch_spread = pitches.std()

    score = pitch_spread * (1 - short_penalty)
    return round(float(score), 2)

# ---------- 자동 전처리 ----------
def preprocess(y, sr, rel_db, top_k):
    S = librosa.stft(y, n_fft=2048, hop_length=512)
    mag, phase = np.abs(S), np.angle(S)
    mag_db = librosa.amplitude_to_db(mag, ref=np.max)

    frame_max = mag_db.max(axis=0, keepdims=True)
    mask_db = mag_db >= (frame_max + rel_db)

    freqs = librosa.fft_frequencies(sr=sr, n_fft=2048)[:, None]
    mask_low = freqs > 40
    mask_high = freqs > 3000

    final_mask = np.zeros_like(mag, dtype=bool)
    for t in range(mag.shape[1]):
        idx = np.argsort(mag[:, t])[-top_k:]
        final_mask[idx, t] = True

    final_mask = (final_mask & mask_db & mask_low) | mask_high
    S_clean = (mag * final_mask) * np.exp(1j * phase)
    return librosa.istft(S_clean, hop_length=512)

# ---------- REL_DB & TOP_K 자동 탐색 ----------
def auto_params(y, sr):
    best = (-1, None, None)
    for rdb in [-30, -35, -40, -45, -50]:
        for k in [6, 8, 10, 12, 14]:
            yc = preprocess(y, sr, rdb, k)
            sf.write("tmp.wav", yc, sr)
            seq = infer_util.predict_sequence(
                estimator=estimator,
                input_fn=lambda: infer_util.input_fn(["tmp.wav"]),
                yield_single_sequence=True
            )
            midi_io.sequence_proto_to_midi_file(seq, "tmp.mid")
            s = score_midi("tmp.mid")
            if s > best[0]:
                best = (s, rdb, k)
    return best[1], best[2], best[0]

# ---------- 메인 처리 ----------
def run_pipeline(audio_file):
    y, sr = librosa.load(audio_file, sr=16000, mono=True)

    rel_db, top_k, quality = auto_params(y, sr)

    y_clean = preprocess(y, sr, rel_db, top_k)
    sf.write("clean.wav", y_clean, sr)

    seq = infer_util.predict_sequence(
        estimator=estimator,
        input_fn=lambda: infer_util.input_fn(["clean.wav"]),
        yield_single_sequence=True
    )

    midi_io.sequence_proto_to_midi_file(seq, "output.mid")

    return "output.mid", f"REL_DB={rel_db}, TOP_K={top_k}, 품질점수={quality}"

# ---------- Gradio ----------
ui = gr.Interface(
    fn=run_pipeline,
    inputs=gr.Audio(type="filepath", label="피아노 mp3 업로드"),
    outputs=[
        gr.File(label="MIDI 다운로드"),
        gr.Text(label="자동 평가 결과")
    ],
    title="🎹 mp3 → MIDI 자동 전사 (Onsets & Frames + AI 전처리)",
    description="REL_DB / TOP_K 자동 최적화 + MIDI 품질 평가 포함"
)

ui.launch(share=True)